In [1]:
# basert på https://www.kaggle.com/airalex/recommend-movie-with-clustering

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn

data = pd.read_csv("../datasets/imdb-5000-movie-dataset/movie_metadata.csv")

data_use = data.ix[:,['genres','plot_keywords','movie_title','actor_1_name',
                      'actor_2_name','actor_3_name','director_name','imdb_score']]

data_use['movie_title'] = [i.replace("\xc2\xa0","") for i in list(data_use['movie_title'])]

In [3]:
data_use.head(5)

,genres,plot_keywords,movie_title,actor_1_name,actor_2_name,actor_3_name,director_name,imdb_score
0,Action|Adventure|Fantasy|Sci-Fi,avatar|future|marine|native|paraplegic,Avatar,CCH Pounder,Joel David Moore,Wes Studi,James Cameron,7.9
1,Action|Adventure|Fantasy,goddess|marriage ceremony|marriage proposal|pi...,Pirates of the Caribbean: At World's End,Johnny Depp,Orlando Bloom,Jack Davenport,Gore Verbinski,7.1
2,Action|Adventure|Thriller,bomb|espionage|sequel|spy|terrorist,Spectre,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Sam Mendes,6.8
3,Action|Thriller,deception|imprisonment|lawlessness|police offi...,The Dark Knight Rises,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Christopher Nolan,8.5
4,Documentary,NaN,Star Wars: Episode VII - The Force Awakens ...,Doug Walker,Rob Walker,NaN,Doug Walker,7.1


In [4]:
print(data_use.shape)
clean_data = data_use.dropna(axis = 0)
print(clean_data.shape)
clean_data = clean_data.drop_duplicates(['movie_title'])
clean_data = clean_data.reset_index(drop=True)
print(clean_data.shape)

(5043, 8)
(4781, 8)
(4659, 8)


In [5]:
people_list = []
for i in range(clean_data.shape[0]):
    name1 = clean_data.ix[i,'actor_1_name'].replace(" ","_")
    name2 = clean_data.ix[i,'actor_2_name'].replace(" ","_")
    name3 = clean_data.ix[i,'actor_3_name'].replace(" ","_")
    name4 = clean_data.ix[i,'director_name'].replace(" ","_")
    people_list.append("|".join([name1,name2,name3,name4]))
clean_data['people'] = people_list



In [6]:
from sklearn.feature_extraction.text import CountVectorizer
def token(text):
    return(text.split("|"))


cv_kw=CountVectorizer(max_features=100,tokenizer=token )
keywords = cv_kw.fit_transform(clean_data["plot_keywords"])
keywords_list = ["kw_" + i for i in cv_kw.get_feature_names()]

cv_ge=CountVectorizer(tokenizer=token )
genres = cv_ge.fit_transform(clean_data["genres"])
genres_list = ["genres_"+ i for i in cv_ge.get_feature_names()]

cv_pp=CountVectorizer(max_features=100,tokenizer=token )
people = cv_pp.fit_transform(clean_data["people"])
people_list = ["pp_"+ i for i in cv_pp.get_feature_names()]

cluster_data = np.hstack([keywords.todense(),genres.todense(),people.todense()*2])
criterion_list = keywords_list+genres_list+people_list

In [7]:
print cluster_data.shape
print  keywords_list

(4659L, 224L)
[u'kw_actor', u'kw_african american', u'kw_alien', u'kw_assassin', u'kw_baby', u'kw_bank', u'kw_bar', u'kw_based on novel', u'kw_battle', u'kw_best friend', u'kw_birthday', u'kw_blood', u'kw_book', u'kw_box office flop', u'kw_boy', u'kw_breasts', u'kw_christmas', u'kw_cia', u'kw_coach', u'kw_college', u'kw_critically bashed', u'kw_cult film', u'kw_death', u'kw_desert', u'kw_detective', u'kw_divorce', u'kw_doctor', u'kw_dog', u'kw_dream', u'kw_drugs', u'kw_escape', u'kw_family relationships', u'kw_fbi', u'kw_female nudity', u'kw_female protagonist', u'kw_fight', u'kw_friend', u'kw_friendship', u'kw_future', u'kw_gang', u'kw_gangster', u'kw_gay', u'kw_ghost', u'kw_girl', u'kw_high school', u'kw_hospital', u'kw_hotel', u'kw_island', u'kw_king', u'kw_lawyer', u'kw_love', u'kw_magic', u'kw_male frontal nudity', u'kw_male nudity', u'kw_marriage', u'kw_martial arts', u'kw_mexico', u'kw_military', u'kw_money', u'kw_monster', u'kw_murder', u'kw_neighbor', u'kw_new york', u'kw_new 

In [8]:
print keywords.todense().shape
print genres.todense().shape
print people.todense().shape

(4659L, 100L)
(4659L, 24L)
(4659L, 100L)


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

def token(text):
    return(text.split("|"))


cv_kw=CountVectorizer(max_features=100,tokenizer=token )
keywords = cv_kw.fit_transform(clean_data["plot_keywords"])
keywords_list = ["kw_" + i for i in cv_kw.get_feature_names()]

cv_ge=CountVectorizer(tokenizer=token )
genres = cv_ge.fit_transform(clean_data["genres"])
genres_list = ["genres_"+ i for i in cv_ge.get_feature_names()]

cv_pp=CountVectorizer(max_features=100,tokenizer=token )
people = cv_pp.fit_transform(clean_data["people"])
people_list = ["pp_"+ i for i in cv_pp.get_feature_names()]

cluster_data = np.hstack([keywords.todense(),genres.todense(),people.todense()*2])
criterion_list = keywords_list+genres_list+people_list

In [10]:
from sklearn.cluster import KMeans

mod = KMeans(n_clusters=100)
category = mod.fit_predict(cluster_data)

category_dataframe = pd.DataFrame({"category":category},index = clean_data['movie_title'])

In [11]:
clean_data.ix[list(category_dataframe['category'] == 0),['genres','movie_title','people']]

,genres,movie_title,people
164,Action|Crime|Thriller,Lethal Weapon 4,Jet_Li|Rene_Russo|Darlene_Love|Richard_Donner
224,Action|Crime|Sci-Fi|Thriller,RoboCop,Gary_Oldman|Abbie_Cornish|Jennifer_Ehle|José_P...
259,Action|Crime|Romance|Thriller,Speed 2: Cruise Control,Jason_Patric|Temuera_Morrison|Lois_Chiles|Jan_...
303,Action|Crime|Fantasy|Romance|Thriller,Catwoman,Frances_Conroy|Christopher_Heyerdahl|Alex_Bors...
308,Action|Crime|Sport|Thriller,Point Break,Ray_Winstone|Edgar_Ramírez|Delroy_Lindo|Ericso...
404,Action|Crime|Thriller,The Fast and the Furious: Tokyo Drift,Amber_Stevens_West|Zachery_Ty_Bryan|Nikki_Grif...
454,Action|Crime|Mystery|Romance|Thriller,Conspiracy Theory,Julia_Roberts|Alex_McArthur|Michael_Potts|Rich...
467,Action|Crime|Thriller,Swordfish,Hugh_Jackman|Don_Cheadle|Sam_Shepard|Dominic_Sena
498,Action|Crime|Thriller,2 Fast 2 Furious,Paul_Walker|Cole_Hauser|Mo_Gallini|John_Singleton
568,Crime|Thriller,Ransom,Lili_Taylor|Delroy_Lindo|Rene_Russo|Ron_Howard


In [12]:
def recommend(movie_name,recommend_number = 5):
    if movie_name in list(clean_data['movie_title']):
        movie_cluster = category_dataframe.ix[movie_name,'category']
        score = clean_data.ix[list(category_dataframe['category'] == movie_cluster),['imdb_score','movie_title']]
        sort_score = score.sort_values(['imdb_score'],ascending=[0])
        sort_score = sort_score[sort_score['movie_title'] != movie_name]
        recommend_number = min(sort_score.shape[0],recommend_number)
        recommend_movie = list(sort_score.iloc[range(recommend_number),1])
        print(recommend_movie)
    else:
        print("Can't find this movie!")

In [18]:
recommend('Toy Story 3',10)

['Spirited Away', 'Princess Mononoke', 'Inside Out', 'Toy Story', 'How to Train Your Dragon', "Howl's Moving Castle", 'The Wizard of Oz', 'The Princess Bride', 'Ratatouille', 'Aladdin']


In [17]:
list(clean_data['movie_title'])

KeyError: 'Toy Story 3'

## TODO :
kanskje plukke ut basert på hvilke som er nærmest cluster senter? Eller er imdb score like greit? Kanskje litt visualisering?